In [1]:
import os
from pm4py.objects.log.importer.xes import factory as xes_importer
import pandas as pd
import numpy as np

1) Set of all tasks

In [4]:
log_tab = [[0, 1, 6, 7, 9, 10, 8, 11],[0, 2, 3, 4, 5, 6, 9, 7, 8, 10, 11]]
max_task = 0

for row in log_tab:
    if (max(row) > max_task):
        max_task = max(row)

print(log_tab)

[[0, 1, 6, 7, 9, 10, 8, 11], [0, 2, 3, 4, 5, 6, 9, 7, 8, 10, 11]]


2) Set of input tasks

In [5]:
Tinput = []

for i in range(len(log_tab)):
    if log_tab[i][0] not in Tinput:
        Tinput.append(log_tab[i][0])
        
print(Tinput)

[0]


3) Set of output tasks

In [6]:
Toutput = []

for i in range(len(log_tab)):
    if log_tab[i][len(log_tab[i])-1] not in Toutput:
        Toutput.append(log_tab[i][len(log_tab[i])-1])
        
print(Toutput)

[11]


4 a) Set of potencial task connections a->b

In [7]:
patternA_list = []
i = -1

for row in log_tab:
    i = i + 1
    for j in range(len(row)-1):
        tupl = (log_tab[i][j], log_tab[i][j+1])
        if tupl not in patternA_list:
            patternA_list.append(tupl)

print(patternA_list)

[(0, 1), (1, 6), (6, 7), (7, 9), (9, 10), (10, 8), (8, 11), (0, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 9), (9, 7), (7, 8), (8, 10), (10, 11)]


* Set of potencial task connections b->a

In [8]:
reverse_patternA_list = []
i = -1

for row in log_tab:
    i = i + 1
    for j in range(len(row)-1):
        tupl = (log_tab[i][len(row)-1-j], log_tab[i][len(row)-2-j])
        if tupl not in reverse_patternA_list:
            reverse_patternA_list.append(tupl)

print(reverse_patternA_list)

[(11, 8), (8, 10), (10, 9), (9, 7), (7, 6), (6, 1), (1, 0), (11, 10), (10, 8), (8, 7), (7, 9), (9, 6), (6, 5), (5, 4), (4, 3), (3, 2), (2, 0)]


Creating footprint:
    #  = 0
    -> = 1
    <- = 2
    || = 3

In [9]:
footprint = np.zeros((max_task+1, max_task+1))

for i in patternA_list:
    x = i[0]
    y = i[1]
    footprint[x][y] = 1

for i in reverse_patternA_list:
    x = i[0]
    y = i[1]
    footprint[x][y] = footprint[x][y] + 2

print(footprint)

[[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 2. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 1. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 3. 1.]
 [0. 0. 0. 0. 0. 0. 2. 3. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 3. 2. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0.]]


4 b) Finding all triples for which a->(b#c)

* potencial triples of pattern B

In [10]:
patternB_list = []

for i in range(len(footprint)):
    tab_tmp = []
    for j in range(len(footprint)):
        if(footprint[i][j] == 1):
            tab_tmp.append(j)
    if(len(tab_tmp) > 1):
        tab_tmp.append(i)
        tab_tmp = np.asarray(tab_tmp)
        patternB_list.append(tab_tmp)
       
print(patternB_list)     # last element is a number of row, it should be actually e.g. ([1,2,5], 0)    

[array([1, 2, 0]), array([7, 9, 6])]


* appropriate triples of pattern B

In [11]:
final_patternB_list = []

for i in range(len(patternB_list)):
    for j in range(len(patternB_list[i])-2):
        for k in range(j+1,len(patternB_list[i])-1):
            x = patternB_list[i][j]
            y = patternB_list[i][k]
            final_triples_row = []
            if(footprint[x][y] == 0):
                final_triples_row.append(patternB_list[i][len(patternB_list[i])-1])
                final_triples_row.append(x)
                final_triples_row.append(y)
                if final_triples_row not in final_patternB_list:
                    final_patternB_list.append(final_triples_row)
                    
print(final_patternB_list)     # final_patternB_list is a set of all triples for which a->(b#c), where [a,b,c]

[[0, 1, 2]]


In [12]:
special_tab = []
i = 0

for elem in final_patternB_list[:-1]:
    i = i + 1
    for elem2 in final_patternB_list[i:]:
        if(elem[0] == elem2[0]):
            if elem not in special_tab:
                special_tab.append(elem)
            if elem2 not in special_tab:
                special_tab.append(elem2)
            
print(special_tab)

[]


4 c) Finding all triples for which (b#c)->d

* potencial triples of pattern C

In [13]:
patternC_list = []

for i in range(len(footprint)):
    tab_tmp2 = []
    for j in range(len(footprint)):
        if(footprint[j][i] == 1):
            tab_tmp2.append(j)
    if(len(tab_tmp2) > 1):
        tab_tmp2.append(i)
        tab_tmp2 = np.asarray(tab_tmp2)
        patternC_list.append(tab_tmp2)
       
print(patternC_list)     # last element is a number of row, it should be actually e.g. ([0,7], 1)    

[array([1, 5, 6]), array([ 8, 10, 11])]


* appropriate triples of pattern C

In [14]:
final_patternC_list = []

for i in range(len(patternC_list)):
    for j in range(len(patternC_list[i])-2):
        for k in range(j+1,len(patternC_list[i])-1):
            x = patternC_list[i][j]
            y = patternC_list[i][k]
            final_triples_row = []
            if(footprint[x][y] == 0):             
                final_triples_row.append(x)
                final_triples_row.append(y)
                final_triples_row.append(patternC_list[i][len(patternC_list[i])-1])
                if final_triples_row not in final_patternC_list:
                    final_patternC_list.append(final_triples_row)

print(final_patternC_list)     # final_patternC_list is a set of all triples for which (b#c)->d, where [b,c,d]

[[1, 5, 6]]


In [15]:
i = 0
for elem in final_patternC_list[:-1]:
    i = i + 1
    for elem2 in final_patternC_list[i:]:
            if(elem[2] == elem2[2]):
                #print(elem[2], elem2[2])
                print("ok")
                if elem not in special_tab:
                    special_tab.append(elem)
                if elem2 not in special_tab:
                    special_tab.append(elem2)
            
print(special_tab)

[]


5) Eliminating doubled tuples

a) eliminating a->b, a->c if there exists some a->(b#c)

In [16]:
for i in range(len(final_patternB_list)):
    tupl = (final_patternB_list[i][0], final_patternB_list[i][1])
    tupl2 = (final_patternB_list[i][0], final_patternB_list[i][2])
    if tupl in patternA_list:
        patternA_list.remove(tupl)
    if tupl2 in patternA_list:
        patternA_list.remove(tupl2)  

print(patternA_list)

[(1, 6), (6, 7), (7, 9), (9, 10), (10, 8), (8, 11), (2, 3), (3, 4), (4, 5), (5, 6), (6, 9), (9, 7), (7, 8), (8, 10), (10, 11)]


b) eliminating b->d, c->d if there exists some (b#c)->d

In [17]:
for i in range(len(final_patternC_list)):
    tupl = (final_patternC_list[i][0], final_patternC_list[i][2])
    tupl2 = (final_patternC_list[i][1], final_patternC_list[i][2])
    if tupl in patternA_list:
        patternA_list.remove(tupl)
    if tupl2 in patternA_list:
        patternA_list.remove(tupl2)  

print(patternA_list)

[(6, 7), (7, 9), (9, 10), (10, 8), (8, 11), (2, 3), (3, 4), (4, 5), (6, 9), (9, 7), (7, 8), (8, 10), (10, 11)]


c) eliminating connections cosed by ||

In [18]:
for i in range(len(footprint)):
    for j in range(len(footprint)):
        if(footprint[i][j] == 3):
            patternA_list.remove((i, j))

print(patternA_list)

[(6, 7), (9, 10), (8, 11), (2, 3), (3, 4), (4, 5), (6, 9), (7, 8), (10, 11)]


d) Finding parallel connections

In [19]:
first_tuple_tab = []
patternD_list = []
patternE_list = []
i = 0
for elem in patternA_list:
    i = i+1
    tmp_tab = []
    tmp_tab2 = []
    for elem2 in patternA_list[i:]:
        if(elem[0] == elem2[0]):
            tmp_tab.append(elem[0])
            tmp_tab.append(elem[1])
            tmp_tab.append(elem2[1])
            patternD_list.append(tmp_tab)
        if(elem[1] == elem2[1]):
            tmp_tab2.append(elem[0])
            tmp_tab2.append(elem2[0])
            tmp_tab2.append(elem[1])
            patternE_list.append(tmp_tab2)   
                                
print(patternD_list)    
print(patternE_list)  

[[6, 7, 9]]
[[8, 10, 11]]


* eliminating connections cosed by a||(b, c)

In [20]:
for i in range(len(patternD_list)):
    tupl = (patternD_list[i][0], patternD_list[i][1])
    tupl2 = (patternD_list[i][0], patternD_list[i][2])
    if tupl in patternA_list:
        patternA_list.remove(tupl)
    if tupl2 in patternA_list:
        patternA_list.remove(tupl2)  

print(patternA_list)

[(9, 10), (8, 11), (2, 3), (3, 4), (4, 5), (7, 8), (10, 11)]


* eliminating connections cosed by (b,c)||d

In [21]:
for i in range(len(patternE_list)):
    tupl = (patternE_list[i][0], patternE_list[i][2])
    tupl2 = (patternE_list[i][1], patternE_list[i][2])
    if tupl in patternA_list:
        patternA_list.remove(tupl)
    if tupl2 in patternA_list:
        patternA_list.remove(tupl2)  

print(patternA_list)

[(9, 10), (2, 3), (3, 4), (4, 5), (7, 8)]


* some addictional necessary work

In [22]:
tab_of_tasks = []

for elem in range(max_task):
    if elem not in Tinput:
        if elem not in Toutput:
            tab_of_tasks.append(elem)
            
print(tab_of_tasks)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [23]:
print(order_tab)

NameError: name 'order_tab' is not defined

In [ ]:
print(Tinput)

In [24]:
print(Toutput)

[11]


Final set of a->b connections (patternA_list)

In [25]:
print(patternA_list)

[(9, 10), (2, 3), (3, 4), (4, 5), (7, 8)]


Final set of a->(b#c) connections (final_triples)

In [26]:
print(final_patternB_list) 

[[0, 1, 2]]


Final set of (b#c)->d connections (final_patternC_list)

In [27]:
print(final_patternC_list) 

[[1, 5, 6]]


Final set of a||(b,c)

In [28]:
print(patternD_list)

[[6, 7, 9]]


Final set of (b,c)||d

In [29]:
print(patternE_list)  

[[8, 10, 11]]


# Drawing a model

In [30]:
import bpmn_python.bpmn_diagram_rep as diagram
import bpmn_python.bpmn_diagram_layouter as layouter
import bpmn_python.bpmn_diagram_visualizer as visualizer
import bpmn_python.bpmn_diagram_rep as diagram
import graphviz
 
bpmn_graph = diagram.BpmnDiagramGraph()
bpmn_graph.create_new_diagram_graph(diagram_name="diagram1")
process_id = bpmn_graph.add_process_to_diagram()

In [31]:
# beginning  TODO : case of multiple elements in Tinput
tab = {}
tab_help=[]

[start_id, _] = bpmn_graph.add_start_event_to_diagram(process_id, start_event_name="start_event")
if(len(Tinput) > 1):  
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for elem in Tinput:
        if elem not in tab_help:
            tab[elem] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem)
            tab_help.append(elem)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, start_id, exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[Tinput[0]][0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[Tinput[1]][0])
else:
    tab[Tinput[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=Tinput[0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, start_id, tab[Tinput[0]][0])
    tab_help.append(Tinput[0])
    
# connections a#(b,c)
for elem in final_patternB_list:
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)  
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[elem[1]][0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[elem[2]][0])     
 
# tasks
for elem in patternA_list:
    if(elem[0] not in tab_help):
        tab[elem[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem[0])
        tab_help.append(elem[0])
    if(elem[1] not in tab_help):
        tab[elem[1]] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem[1])
        tab_help.append(elem[1])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], tab[elem[1]][0]) 

# connections (b,c)#d
for elem in final_patternC_list:
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[1]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[elem[2]][0])
               
# connections (b,c)||d
for elem in patternE_list:
    [plus_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="+")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], plus_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[1]][0], plus_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, plus_gate_id, tab[elem[2]][0])           
        
# connections a||(b,c)
for elem in patternD_list:
    [plus_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="+")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)   
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], plus_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, plus_gate_id, tab[elem[1]][0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, plus_gate_id, tab[elem[2]][0])  

# end  TODO : case of multiple elements in Tinput
if(len(Toutput) > 1):
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for elem in Toutput:
        if elem not in tab_help: 
            tab[elem] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem)
            tab_help.append(elem)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[0]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[1]][0], exclusive_gate_id)
    [end_id, _] = bpmn_graph.add_end_event_to_diagram(process_id, end_event_name="end_event")
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, end_id)
else:
    if Toutput[0] not in tab_help:
        tab[Toutput[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=Toutput[0])
    [end_id, _] = bpmn_graph.add_end_event_to_diagram(process_id, end_event_name="end_event")
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[0]][0], end_id)       
    
visualizer.bpmn_diagram_to_png(bpmn_graph, file_name = 'model_example')